In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
import os 

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType

StatementMeta(, f59feb8d-1523-4b5e-862c-b2a7d0c41f86, 3, Finished, Available, Finished)

In [2]:
# Iniciar Spark (en Fabric normalmente ya está disponible como spark)
spark = SparkSession.builder.getOrCreate()

StatementMeta(, f59feb8d-1523-4b5e-862c-b2a7d0c41f86, 4, Finished, Available, Finished)

In [3]:
headers = {
    "Accept": "application/json; application/vnd.esios-api-v1+json",
    "Content-Type": "application/json",
    "x-api-key": "273695bb4f30b8ab8de054a0410ee2a9d09c97a5006e3a07c27615e6c60888a7"
}

StatementMeta(, f59feb8d-1523-4b5e-862c-b2a7d0c41f86, 5, Finished, Available, Finished)

In [4]:
# 📌 Indicadores con sus nombres
indicators = [
    {"type": "hidraulica", "id": "10330"},
    {"type": "eolica", "id": "10333"},
    {"type": "solar-fotovoltaica", "id": "1707"},
    {"type": "solar-termica", "id": "1708"},
    {"type": "otras-renovables", "id": "10334"},
    {"type": "ciclo-combinado", "id": "1703"},
    {"type": "carbon", "id": "10331"},
    {"type": "cogeneracion", "id": "10335"},
    {"type": "residuos-no-renovables", "id": "10336"},
    {"type": "turbinacion-bombeo", "id": "1694"},
    {"type": "consumo-bombeo", "id": "1721"},
    {"type": "saldo-intercambios", "id": "1722"}
]

# 📆 Rango de fechas por año
date_ranges = {
    "2023": ("01-01-2023T00:00", "31-12-2023T23:55"),
    "2024": ("01-01-2024T00:00", "31-12-2024T23:55"),
    "2025": ("01-01-2025T00:00", "1-10-2025T23:55"),
}

# ⏱️ Granularidades
time_truncs = ["month"] # ["day", "month"] #solo muestra por meses


StatementMeta(, f59feb8d-1523-4b5e-862c-b2a7d0c41f86, 6, Finished, Available, Finished)

In [5]:
# 📂 Carpeta base dentro del Lakehouse (BRONZE)
base_dir = "/lakehouse/default/Files/bronze/ESIOS/data/balance/balance-electrico"

StatementMeta(, f59feb8d-1523-4b5e-862c-b2a7d0c41f86, 7, Finished, Available, Finished)

In [6]:
# 🚀 Loop de descargas
for indicator in indicators:
    for year, (start_date, end_date) in date_ranges.items():
        for trunc in time_truncs:
            url = (
                f"https://api.esios.ree.es/indicators/{indicator['id']}?"
                f"start_date={start_date}&end_date={end_date}&time_trunc={trunc}"
            )
            try:
                response = requests.get(url, headers=headers, timeout=60)
                response.raise_for_status()
                data = response.json()

                # ----------------------
                # 1) Guardar JSON crudo en Files/
                # ----------------------
                filename = f"brz-balance-{indicator['type']}-{trunc}-{year}.json"
                year_dir = os.path.join(base_dir, trunc, str(year))
                os.makedirs(year_dir, exist_ok=True)
                filepath = os.path.join(year_dir, filename)

                with open(filepath, "w", encoding="utf-8") as f:
                    json.dump(data, f, ensure_ascii=False, indent=2)

                print(f"✅ Guardado JSON en Lakehouse: {filepath}")

            except requests.exceptions.RequestException as e:
                print(f"❌ Error en {url}: {e}")

print(f"\nProceso terminado - {datetime.utcnow().isoformat()}Z")

StatementMeta(, f59feb8d-1523-4b5e-862c-b2a7d0c41f86, 8, Finished, Available, Finished)

✅ Guardado JSON en Lakehouse: /lakehouse/default/Files/bronze/ESIOS/data/balance/balance-electrico/month/2023/brz-balance-hidraulica-month-2023.json
✅ Guardado en Tables/brz_balance_hidraulica_month (192 registros)
✅ Guardado JSON en Lakehouse: /lakehouse/default/Files/bronze/ESIOS/data/balance/balance-electrico/month/2024/brz-balance-hidraulica-month-2024.json
✅ Guardado en Tables/brz_balance_hidraulica_month (192 registros)
✅ Guardado JSON en Lakehouse: /lakehouse/default/Files/bronze/ESIOS/data/balance/balance-electrico/month/2025/brz-balance-hidraulica-month-2025.json
✅ Guardado en Tables/brz_balance_hidraulica_month (62 registros)
✅ Guardado JSON en Lakehouse: /lakehouse/default/Files/bronze/ESIOS/data/balance/balance-electrico/month/2023/brz-balance-eolica-month-2023.json
✅ Guardado en Tables/brz_balance_eolica_month (190 registros)
✅ Guardado JSON en Lakehouse: /lakehouse/default/Files/bronze/ESIOS/data/balance/balance-electrico/month/2024/brz-balance-eolica-month-2024.json
✅ Gu